# 1. **Setup The Environment (GPU)**

In [ ]:
import torch
import sys
!pip install yapf==0.31.0
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Check GPU name

# 2. **Setup The Repository**

In [ ]:
!git clone https://github.com/IDEA-Research/DINO.git

2.1 Installing the Dependencies

In [ ]:
!pip install -r DINO/requirements.txt

2.2 Adding the path of files to Environment system so that python can find and run it.

In [ ]:
sys.path.append('/content/DINO/models/dino/ops')

2.3 Running Files to setup the dependencies

In [ ]:
%cd /content/DINO/models/dino/ops
!python setup.py build install

2.4 Testing the setup

In [ ]:
!python test.py

## 3. **Importing Libraries For Further Use**

In [ ]:
import os
import json
import numpy as np
sys.path.append('/content/DINO')
from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops

4. **To Slice the annotation file for train and validation**

In [ ]:
# import json

# # Path to your original annotations file Train/Validation
# annotation_file_path = "/content/DATASET/annotations/instances_val2017.json"
# # Path to save the modified annotations file Train/Validation
# output_file_path = "/content/DATASET/annotations/val.json"

# # Define the categories (adjust as needed)
# categories = [
#     { "id": 1, "name": "person" }
#     # Add more categories here if needed
# ]

# # Load existing annotations
# with open(annotation_file_path, 'r') as file:
#     data = json.load(file)

# # Add the 'categories' field
# data['categories'] = categories

# # Save the modified annotations
# with open(output_file_path, 'w') as file:
#     json.dump(data, file, indent=4)

# print("Added 'categories' field successfully.")


# **5. Evaluation And Visualization**

In [ ]:
 pip install --upgrade pycocotools

In [ ]:
%cd /content/DINO
!bash scripts/DINO_eval.sh /content/DATASET /content/checkpoint0011_4scale.pth

In [ ]:
model_config_path = "/content/DINO/config/DINO/DINO_4scale.py" # change the path of the model config file
model_checkpoint_path = "/content/checkpoint0011_4scale.pth" # change the path of the model checkpoint
# See our Model Zoo section in README.md for more details about our pretrained models.

In [ ]:
sys.path.append('/usr/local/lib/python3.10/dist-packages/MultiScaleDeformableAttention-1.0-py3.10-linux-x86_64.egg')

In [ ]:
model_config_path = "/content/DINO/config/DINO/DINO_4scale.py" # change the path of the model config file
model_checkpoint_path = "/content/checkpoint0011_4scale.pth" # change the path of the model checkpoint
# See our Model Zoo section in README.md for more details about our pretrained models.

args = SLConfig.fromfile(model_config_path)
args.device = 'cuda'
model, criterion, postprocessors = build_model_main(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
_ = model.eval()

In [ ]:
with open('util/coco_id2name.json') as f:
    id2name = json.load(f)
    id2name = {int(k):v for k,v in id2name.items()}

In [ ]:
args.dataset_file = 'coco'
args.coco_path = "/content/DATASET" # the path of coco
args.fix_size = False

dataset_val = build_dataset(image_set='val', args=args)

In [ ]:
image, targets = dataset_val[1]
# build gt_dict for vis
box_label = [id2name[int(item)] for item in targets['labels']]
gt_dict = {
    'boxes': targets['boxes'],
    'image_id': targets['image_id'],
    'size': targets['size'],
    'box_label': box_label,
}
vslzr = COCOVisualizer()
vslzr.visualize(image, gt_dict, savedir=None)

In [ ]:
output = model.cuda()(image[None].cuda())
output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

In [ ]:
thershold = 0.3 # set a thershold

scores = output['scores']
labels = output['labels']
boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
select_mask = scores > thershold

In [ ]:
box_label = [id2name[int(item)] for item in labels[select_mask]]
pred_dict = {
    'boxes': boxes[select_mask],
    'size': targets['size'],
    'box_label': box_label
}
vslzr.visualize(image, pred_dict, savedir=None)

# ***6. Model Training on Pedestarian Dataset ***

 ( After changing some parameters for experiments )

In [ ]:
!bash scripts/DINO_train.sh /content/DATASET